In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import yaml
from src.antares_io import (
    load_api_config_from_yaml,
    get_or_create_study,
    update_study_id_in_config,
    load_run_config,
    load_input_datasets
)
from src.antares_updater import AntaresStudyUpdater
from pommes.io.build_input_dataset import build_input_parameters, read_config_file
from pommes.model.data_validation.dataset_check import check_inputs
import pandas as pd
import antares.craft as ac
import datetime



In [2]:
CONFIG_LOCAL_PATH = Path("config_local.yaml")
CONFIG_RUN_PATH = Path("config_run.yaml")
CONFIG_ANTARES_PATH = Path("config_antares.yaml")

# Load all configurations
local_config = yaml.safe_load(open(CONFIG_LOCAL_PATH))
run_config = load_run_config(CONFIG_RUN_PATH)
antares_config = yaml.safe_load(open(CONFIG_ANTARES_PATH)) # For UI, etc.

# Load all data based on run config
datasets = load_input_datasets(run_config)

# Load POMMES model parameters (still needed for costs, efficiencies etc.)
# TODO This part might also be simplified in the future, likely by loading the pommes netcdf model input file
if True:
    scenario = run_config['pommes']['scenario']
    weather_year = run_config['pommes']['weather_year']

    pommes_config = read_config_file(study=scenario, file_path="config_pommes.yaml")
    pommes_config["coords"]["area"]["values"] = run_config['study_parameters']['areas']

    ###File pathway update
    if scenario=="DE":
        pommes_config["input"]["path"]="data/Distributed Energy"
    elif scenario=="GA":
        pommes_config["input"]["path"] = "data/Global Ambition"
    else:
        print(404)
        exit()

    pommes_config["coords"]["area"]["values"]=run_config['study_parameters']['areas']
    pommes_config["coords"]["year_op"]["values"] = run_config['study_parameters']['simulation_year']


    # pommes_config["input"]["parameters"]["demand"]["file"]=f"demand_cy{weather_year}.csv"
    # pommes_config["input"]["parameters"]["conversion_availability"]["file"] = f"availability_cy{weather_year}.csv"
    pommes_config["input"]["parameters"]["conversion_max_yearly_production"]["file"] = f"conversion_op2_cy{weather_year}.csv"
    pommes_config["input"]["parameters"]["conversion_power_capacity_max"]["file"] = f"conversion_op2_cy{weather_year}.csv"
    pommes_config["input"]["parameters"]["conversion_power_capacity_min"]["file"] = f"conversion_op2_cy{weather_year}.csv"

    ####Links adjustment
    if pommes_config["add_modules"]["transport"] :
        areas=pommes_config["coords"]["area"]["values"]
        all_links=pd.read_csv(pommes_config["input"]["path"]+"/transport_link.csv",sep=";").link.unique()
        links = []

        for link in all_links:
            pos = ""
            i = 0
            while pos != "-":
                pos = link[i]
                i += 1
            area_from = link[:i - 1]
            area_to = link[i:]
            if area_to in areas and area_from in areas:
                links.append(link)
        if len(links) >= 1:
            pommes_config["coords"]["link"]["values"] = links
        else:
            pommes_config["add_modules"]["transport"]=False
    print("Transport activated:", pommes_config["add_modules"]["transport"])
print("\033[1m Building Pommes Model  \033[0m")
model_parameters = build_input_parameters(pommes_config)
model_parameters = check_inputs(model_parameters)


--- Loading All Input Datasets ---
   - INFO: Adding zero-filled data for missing nodes: ['CH00', 'NO00']
All datasets loaded successfully.
Transport activated: True
 Building Pommes Model  


C:\Users\hamburgerhug\AppData\Local\miniconda3\envs\pommes-env\Lib\site-packages\xarray\core\duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [3]:
api_config = load_api_config_from_yaml(CONFIG_LOCAL_PATH)
study_id = local_config.get("antares_study", {}).get("id")
study_name = local_config.get("antares_study", {}).get("name", "default-sl-study-name")
study = get_or_create_study(api_config=api_config, study_id=study_id, study_name=study_name)
if study.service.study_id != study_id:
    update_study_id_in_config(CONFIG_LOCAL_PATH, study.service.study_id)

Proxy configured for proxy-window-rte-france.com:3128
Successfully loaded Antares study 'HH-SL-0.1.1' (ID: f5868019-7181-408f-9679-a7e5879c19f2) from API.


In [ ]:
# Execute the full update workflow
updater.update_areas()
updater.update_virtual_areas()
updater.update_links()
updater.update_thermal_clusters(thermal_tech_map, tech_maps['pommes_to_antares_group'])
updater.update_renewable_clusters(tech_maps['vre'], tech_maps['pommes_to_antares_group'], tech_maps['pommes_to_pecd'])
updater.update_st_storage(st_storage_map, tech_maps['pommes_to_antares_group'], settings['hurdle_cost'])
updater.update_hydro(tech_maps['pommes_to_antares_group'], settings['hydro_policy'])
updater.update_run_of_river()
updater.update_sector_coupling(tech_maps['p2g'], tech_maps['g2p'], tech_maps['pommes_to_antares_group'])

print("\n\033[1mANTARES study update complete!\033[0m")

In [4]:

# --- 1. Initialize the Updater ---

# This single object now holds all the necessary data and the study itself.
updater = AntaresStudyUpdater(
    study=study,
    model_parameters=model_parameters,
    pommes_output=datasets["pommes_output"],
    antares_config=antares_config,
    load_el=datasets["load_el"],
    load_h2=datasets["load_h2"],
    pecd=datasets["pecd"],
    hydro_data=datasets["hydro_data"],
    areas=run_config['study_parameters']['areas'],
    weather_years=list(range(
        run_config['study_parameters']['weather_years']['start'],
        run_config['study_parameters']['weather_years']['end'] + 1
    )),
    year=run_config['study_parameters']['simulation_year']
)

# Get all settings and mappings directly from the loaded config
tech_maps = run_config['technology_mappings']
settings = run_config['model_settings']


thermal_tech_map = {'electricity': tech_maps['thermal_el'], 'hydrogen': tech_maps['thermal_h2']}
st_storage_map = {'electricity': tech_maps['st_storage_el'], 'hydrogen': tech_maps['st_storage_h2']}

AntaresStudyUpdater initialized for study 'HH-SL-0.1.1' and year 2040.


In [6]:
study.get_outputs()
study.get_output()

mappingproxy({'20250911-1451eco': <antares.craft.model.output.Output at 0x2e637ab8490>,
              '20250911-1523eco': <antares.craft.model.output.Output at 0x2e637ab85d0>,
              '20250911-1556eco': <antares.craft.model.output.Output at 0x2e637ad0ad0>})

In [5]:
api_config = load_api_config_from_yaml(CONFIG_LOCAL_PATH)
study_id = local_config.get("antares_study", {}).get("id")
study_name = local_config.get("antares_study", {}).get("name", "default-sl-study-name")
study = get_or_create_study(api_config=api_config, study_id=study_id, study_name=study_name)
if study.service.study_id != study_id:
    update_study_id_in_config(CONFIG_LOCAL_PATH, study.service.study_id)

import antares.craft as ac
# TEST HYDRO_FAST, 34 WY, PRESOLVE -------  LAST RUN -------------  // TESTER SIRIUS
solver = "xpress"
if solver == "xpress":
    solver = ac.Solver.XPRESS # sirius, xpress, gurobi (in 9.3)
nb_cpu = run_config['study_parameters']['weather_years']['end'] - run_config['study_parameters']['weather_years']['start'] +1
simulation_parameters = ac.AntaresSimulationParameters(
    solver=solver,
    nb_cpu=nb_cpu,
    presolve=True,
)
print(f"start - {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
job = study.run_antares_simulation(simulation_parameters)
study.wait_job_completion(job)
print(f"end - {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Proxy configured for proxy-window-rte-france.com:3128
Successfully loaded Antares study 'HH-SL-0.1.1' (ID: f5868019-7181-408f-9679-a7e5879c19f2) from API.
start - 2025-09-11 15:54:51
end - 2025-09-11 15:59:22


In [9]:
simulation_parameters.to_api()
ac.Solver.XPRESS.value

'xpress'

In [28]:
import antares.craft as ac
import datetime
import time

# --- 1. Define Simulation Parameters ---
nb_cpu = run_config['study_parameters']['weather_years']['end'] - run_config['study_parameters']['weather_years']['start'] + 1
solver_name_for_api = "xpress"  # <-- CRITICAL CHANGE: Use lowercase for AntaREST compatibility

# --- 2. Manually Construct the API Payload with the Corrected Solver String ---
print(f"--- Preparing Simulation (Testing lowercase solver: '{solver_name_for_api}') ---")

# Start with the base parameter object
base_parameters = ac.AntaresSimulationParameters(nb_cpu=nb_cpu)
api_payload = base_parameters.to_api()

# This is the crucial step: Add the 'other_options' key with the lowercase value.
api_payload['other_options'] = solver_name_for_api
print(f"Final API payload being sent: {api_payload}")


# --- 4. Launch Simulation and Wait ---
job_id = None
output_id = None
try:
    print(f"Submitting job at: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Use the service layer to send our custom payload
    url = f"{study.service._base_url}/launcher/run/{study_id}"
    response = study.service._wrapper.post(url, json=api_payload)
    response.raise_for_status()

    job_id = response.json()["job_id"]
    print(f"Job '{job_id}' submitted successfully. Now waiting for completion...")

    while True:
        job_status_obj = study.get_job_status(job_id)
        current_status = job_status_obj.status
        print(f"  Current job status: {current_status.value}...")
        if current_status in [ac.JobStatus.SUCCESS, ac.JobStatus.FAILED]:
            break
        time.sleep(15)

    print(f"Job completed at: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # --- 5. Check Final Status and Get Results ---
    final_job_status = study.get_job_status(job_id)

    if final_job_status.status == ac.JobStatus.SUCCESS:
        output_id = final_job_status.output_id
        print("\n\033[1mSimulation successful!\033[0m")
        print(f"Output ID: {output_id}")
        run_config['last_output_id'] = output_id
    else:
        print(f"\n\033[1mError: Simulation failed with status '{final_job_status.status.value}'.\033[0m")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

# --- 6. (Optional) Fetch Logs on Failure ---
if output_id is None and job_id:
    try:
        logs = study.get_job_logs(job_id)
        print("--- SIMULATION LOGS ---\n", logs.get('both', 'No logs available.'), "\n--- END OF LOGS ---")
    except Exception as log_e:
        print(f"Could not retrieve job logs: {log_e}")

--- Preparing Simulation (Testing lowercase solver: 'xpress') ---
Final API payload being sent: {'nb_cpu': 34, 'auto_unzip': True, 'other_options': 'xpress'}
Deleting previous simulation outputs...
Submitting job at: 2025-09-11 14:48:58
Job '0002ed20-34ac-4b40-9288-742846cb5414' submitted successfully. Now waiting for completion...
An unexpected error occurred: 'Study' object has no attribute 'get_job_status'
Could not retrieve job logs: 'Study' object has no attribute 'get_job_logs'


In [ ]:
#TODO UNFEASIBILITY WITH ACCURATE MODE (not sure) LAUNCHED BY THE API

In [ ]:
#TODO study options, e.g. hydro accurate ...
updater.study.set_playlist()
updater.study.set_thematic_trimming()
updater.study.set_scenario_builder()
updater.study.generate_thermal_timeseries()


In [ ]:


#TODO: use a seed for the base study, variant for each year 2030, 2040, 2050
updater.study.create_variant()
updater.study.delete()
updater.study.run_antares_simulation()


In [ ]:
# TODO: Add a cell to run study
# TODO: Add a cell to read results
# TODO: Filter outputs saved
# TODO delete outputs

# TODO: Add h2 import
# TODO: add salt_cavern
# todo forward availability for nuke / old_nuke conversion_tech
# todo forward mustrun for nuke / old_nuke conversion_tech  -->> Defined per cluster in antares, not possible here
# TODO:
# TODO: add parameters not yet implemented

# TODO: propper logger
# TODO: skip part of the processes that are not necessary  after the study has been initialised (e.g. it 3, no need to send VRES load_factor (defined in % and so normalised) again, as they are already properly defined and will not be affected by invenstment changes, likely the same for hydro as they can't be invested
# TODO: check if it works for 2030, 2040 and 2050
# TODO: read directly pommes inputs file


Antares study ID:
* fd9aef41-4386-455d-9d69-39123b9d1360 # Old one
* d6f3cd34-fec2-4844-8185-f5ed78ebddb7 # ID study creation issue due to no output folder
* d1677c86-4938-49bb-b4be-acd9afa3fb65